In [19]:
!pip install -U scikit-learn scipy matplotlib tensorflow scikeras


In [33]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

cf10_classes = ['airplan','auto','bird','cat','deer','dog','frog','horse','ship','truck']
num_classes = len(cf10_classes)

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
print("Original train size = ",X_train.shape)
print("Original test size = ",X_test.shape)

x_train, x_test, y_train, y_test=train_test_split(X_train, Y_train, test_size=0.90, random_state=42, shuffle=True)
x_test, x_valid, y_test, y_valid=train_test_split(x_test, y_test, test_size=0.90, random_state=42, shuffle=True)
print("Train size = ",x_train.shape)
print("Test size = ",x_test.shape)
print("Valid size = ",x_valid.shape)

x_train = x_train/255
x_test = x_test/255
x_valid = x_valid/255

y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)
y_valid = to_categorical(y_valid,num_classes)



Original train size =  (50000, 32, 32, 3)
Original test size =  (10000, 32, 32, 3)
Train size =  (5000, 32, 32, 3)
Test size =  (4500, 32, 32, 3)
Valid size =  (40500, 32, 32, 3)


In [43]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model

input_shape=x_train.shape[1:]
print ("The input shape: ",input_shape)

def make_model ():
    model = Sequential()
    model.add (Conv2D(filters=32,kernel_size=(3,3), input_shape=input_shape))
    model.add (Activation('relu'))
    model.add (Conv2D(filters=24,kernel_size=(3,3)))
    model.add (Activation('relu'))
    model.add (MaxPooling2D(pool_size=(2,2)))
    model.add (Dropout(0.1))

    model.add (Conv2D(filters=16,kernel_size=(3,3)))
    model.add (Activation('relu'))
    model.add (MaxPooling2D(pool_size=(2,2)))
    model.add (Dropout(0.1))

    model.add (Flatten())
    model.add (Dense(100))
    model.add (Activation('relu'))
    model.add (Dropout(0.1))
    model.add (Dense(num_classes))
    model.add (Activation('softmax'))
    return model;

model =make_model()
model.summary()

The input shape:  (32, 32, 3)
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 activation_35 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 conv2d_22 (Conv2D)          (None, 28, 28, 24)        6936      
                                                                 
 activation_36 (Activation)  (None, 28, 28, 24)        0         
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 14, 14, 24)        0         
 ng2D)                                                           
                                                                 
 dropout_23 (Dropout)        (None, 14, 14, 24)        0         
                        

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
hist = model.fit(x_train,y_train,epochs=500,batch_size=100,validation_data=(x_test,y_test),verbose=1)

Epoch 1/500
50/50 [==============================] - 5s 92ms/step - loss: 2.3050 - accuracy: 0.1000 - val_loss: 2.3006 - val_accuracy: 0.1160
Epoch 2/500
50/50 [==============================] - 4s 86ms/step - loss: 2.2995 - accuracy: 0.1138 - val_loss: 2.2983 - val_accuracy: 0.1420
Epoch 3/500
50/50 [==============================] - 4s 86ms/step - loss: 2.2961 - accuracy: 0.1210 - val_loss: 2.2946 - val_accuracy: 0.1553
Epoch 4/500
50/50 [==============================] - 4s 86ms/step - loss: 2.2929 - accuracy: 0.1306 - val_loss: 2.2894 - val_accuracy: 0.1624
Epoch 5/500
50/50 [==============================] - 4s 86ms/step - loss: 2.2876 - accuracy: 0.1432 - val_loss: 2.2825 - val_accuracy: 0.1544
Epoch 6/500
50/50 [==============================] - 4s 86ms/step - loss: 2.2774 - accuracy: 0.1496 - val_loss: 2.2703 - val_accuracy: 0.1451
Epoch 7/500
50/50 [==============================] - 4s 85ms/step - loss: 2.2658 - accuracy: 0.1574 - val_loss: 2.2543 - val_accuracy: 0.1511
Epoch 

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

def make_compiled_model (loss='categorical_crossentropy',optimizer='sgd',metrics='accuracy') :
    model = make_model();
    model.compile(loss=loss,optimizer=optimizer,metrics=[metrics])
    return model

def print_results (grid_result):
    print("  --> Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']

    for mean, stdev, param in zip(means, stds, params):
        print("      > %f (%f) with: %r" % (mean, stdev, param))

keras_classifier = KerasClassifier(model=make_compiled_model, verbose=1)

batch_size = [100, 500, 1000]
epochs = [100, 200, 300]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=keras_classifier, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)
grid_result = grid.fit(x_train, y_train)

print_results(grid_result);

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Epoch 1/300
Epoch 1/200
Epoch 1/100
Epoch 1/300
Epoch 1/200
Epoch 1/100
Epoch 1/100
Epoch 1/200
34/34 [==============================] - 17s 471ms/step - loss: 2.3054 - accuracy: 0.1038
Epoch 2/300
34/34 [==============================] - 17s 476ms/step - loss: 2.3090 - accuracy: 0.1029
Epoch 2/200
34/34 [==============================] - 17s 476ms/step - loss: 2.3210 - accuracy: 0.1068
Epoch 2/300
34/34 [==============================] - 17s 482ms/step - loss: 2.3085 - accuracy: 0.0999
Epoch 2/100
34/34 [==============================] - 17s 481ms/step - loss: 2.3063 - accuracy: 0.1134
Epoch 2/100
34/34 [==============================] - 17s 481ms/step - loss: 2.3104 - accuracy: 0.1038
Epoch 2/100
34/34 [==============================] - 16s 459ms/step - loss: 2.3019 - accuracy: 0.1062
Epoch 3/200
34/34 [==============================] - 16s 473ms/step - loss: 2.3035 - accuracy: 0.0969
Epoch 3/300
34/34 [=====================

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


34/34 [==============================] - 15s 456ms/step - loss: 1.4815 - accuracy: 0.4695
Epoch 107/300
18/34 [==============>...............] - ETA: 7s - loss: 2.2970 - accuracy: 0.1050

2023-10-02 16:07:01.661480: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.


34/34 [==============================] - 15s 454ms/step - loss: 1.4956 - accuracy: 0.4587
Epoch 106/300
 1/34 [..............................] - ETA: 14s - loss: 1.4266 - accuracy: 0.4900

2023-10-02 16:07:02.119887: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-02 16:07:02.121889: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-02 16:07:02.148054: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-02 16:07:02.219047: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.


34/34 [==============================] - 15s 429ms/step - loss: 1.4755 - accuracy: 0.4671
Epoch 107/200
34/34 [==============================] - 15s 430ms/step - loss: 2.2987 - accuracy: 0.1035
Epoch 4/300
34/34 [==============================] - 15s 435ms/step - loss: 1.4810 - accuracy: 0.4611
Epoch 108/200
34/34 [==============================] - 15s 431ms/step - loss: 1.4793 - accuracy: 0.4739
Epoch 108/200
31/34 [==========================>...] - ETA: 1s - loss: 1.4810 - accuracy: 0.46550

2023-10-02 16:07:12.910389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


34/34 [==============================] - 15s 445ms/step - loss: 1.4811 - accuracy: 0.4671
Epoch 108/300
34/34 [==============================] - 15s 438ms/step - loss: 1.4848 - accuracy: 0.4578
Epoch 107/300
34/34 [==============================] - 15s 442ms/step - loss: 1.4787 - accuracy: 0.4674
Epoch 108/200
34/34 [==============================] - 15s 433ms/step - loss: 2.2945 - accuracy: 0.1209
Epoch 5/300
34/34 [==============================] - 15s 436ms/step - loss: 1.4681 - accuracy: 0.4614
Epoch 109/200
34/34 [==============================] - 15s 425ms/step - loss: 1.4826 - accuracy: 0.4709
Epoch 109/200
Epoch 1/100
7/7 [==============================] - 15s 2s/step - loss: 2.2947 - accuracy: 0.1140
Epoch 5/100
34/34 [==============================] - 15s 436ms/step - loss: 1.4656 - accuracy: 0.4716
Epoch 109/300
34/34 [==============================] - 15s 433ms/step - loss: 1.4864 - accuracy: 0.4641
Epoch 108/300
34/34 [==============================] - 15s 433ms/step - los

In [ ]:

keras_classifier = KerasClassifier(model=create_model_without_compile, loss="binary_crossentropy",
                                   epochs=100, batch_size=50, verbose=0)

# define the grid search parameters
optimizer = ['Adadelta','Adafactor','Adagrad','Adam','AdamW','Adamax', 'Ftrl','Lion', 'Nadam','SGD', 'RMSprop']
param_grid = dict(optimizer=optimizer)

grid = GridSearchCV(estimator=keras_classifier, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)

# summarize results
print_results(grid_result);